# Training and exploring the model evaluation

Jupyter notebooks are great for work's orchestration and interactive exploration. However, authoring code in notebooks can quickly become a proactice that is hard to scale. Testing and source control may become challenging.

In this notebook, we will explora how we can combine the two.

## Importing our library

Core functionality for our model is implemented in Python libraries, which allows us to use more robust testing.

The way we designed the library is structure as follows:

* `carpricer.dataprep` contains logic related to data preparation, including transformations.
* `carpricer.train` contains logic related to training and evaluating models.

In [1]:
import carpricer.dataprep as dataprep
from carpricer.dataprep import transformations
from carpricer.train import evaluator, trainer

### Model's configuration and hyperparameters
Our model's configuration is not stored in code, but in a configuration file. The choice we used here is on a `YAML` file. This makes easier to segregate configuration from code. 

Let's have a look at it:

In [2]:
!cat ../src/carpricer.params.yml

model:
  baseline: 
    objective: reg:squarederror
    min_child_weight: 4
    colsample_bytree: .7
    n_estimators: 200
  tune:
    cv: 5
    objective: neg_mean_squared_error
    search:
      learning_rate:
      - .03
      - .05
      - .07
      - .5
      max_depth:
      - 3
      - 7
      - 9
      - 20
      alpha:
      - 0
      - .5
      - 1
      subsample:
      - .6
      - .8
      - 1
data:
  label: lnprice
  test_size: 0.3


### Arguments to use

Let's use the following arguments for this example:

In [3]:
train_path = "../data/car-prices/sample/automobile_prepared.csv"
params_file = "../src/carpricer.params.yml"

The library `jobtools` has functionalities to quickly parse a `YAML` file to a namespace object that can use the "dot notation" to access it's members:

In [4]:
from jobtools.arguments import file2namespace

In [5]:
params = file2namespace(params_file)

## Loading data and preprocessing

Let's use our preprocessing library to read the data

In [6]:
X_train, y_train, X_test, y_test = dataprep.read_and_split(train_path, params.data.test_size, params.data.label)

The method `scale_and_encode` was designed to preprocess a given dataset and also return the transformations fitted to it. This allows us to then store this transformations for when the model is deployed.

In [7]:
X_train_transformed, transforms = transformations.scale_and_encode(X_train)

Let's apply the same transformations to our testing dataset:

In [8]:
X_test_transformed = transforms.transform(X_test)

## Training routine

We will use Mlflow for tracking metrics, parameters and models in our experiments.

In [9]:
import mlflow

In [ ]:
mlflow.start_run()

### The base model

Our base model will be an XGBoost regressor with some basic configuration. 

In [11]:
from xgboost.sklearn import XGBRegressor

We have some specific parameters for this regressor that are stored in the configuration file:

In [12]:
params.model.baseline

namespace(objective='reg:squarederror',
          min_child_weight=4,
          colsample_bytree=0.7,
          n_estimators=200)

We are going to pass this arguemnts to the constructor or the regressor:

In [13]:
base_model = XGBRegressor(silent=True, nthread=4, **params.model.baseline.to_dict())

As a good practice, let's log the parameters used to our experiment:

In [ ]:
mlflow.log_params(params.model.baseline.to_dict()) 

### Searching for hyperparameters

We designed a method to search for hyperparameters `fit_and_optimize`. This method recieves a training dataset, a base model and a grid with all the parameters to look for:

In [ ]:
search = trainer.fit_and_optimize(X_train_transformed, 
                                  y_train, 
                                  base_model=base_model,
                                  param_grid=params.model.tune.search.to_dict(),
                                  cv=params.model.tune.cv,
                                  scoring_fit=params.model.tune.objective)

### Evaluating the results

Searching for the best configuration for the model may require some exploration. We are using the library `sklearn_evaluation` to evaluate results.

`sklearn_evaluation` is used in the `evaluate_search` method where we pass the search results of the tunning and the parameters we are looking to explore:

In [ ]:
evaluator.evaluate_search(search, plot_params_name=['learning_rate', 'max_depth'])

Let's fin the best model, the best parameters combination and log them with Mlflow:

In [ ]:
best_model = search.best_estimator_
best_params = search.best_params_
mlflow.log_params(best_params)

We are going to calculate a couple of metrics for the chosen model:

In [ ]:
metrics = evaluator.evaluate_regressor(best_model, X_test_transformed, y_test)
mlflow.log_metrics(metrics)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


### Constructing the inference pipeline

Now we have a good model at hand, we can construct a pipeline with both the preprocessing step and an estimator. This pipleline can then be stored for inference:

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
model_pipeline = Pipeline(steps=[('preprocessing', transforms),
                                 ('model', best_model)])

#### Logging the model with Mlflow

We are going to log the winning model with mlflow. First let's see which is the expected inputs and outputs of the model:

In [ ]:
from mlflow.models.signature import infer_signature

In [ ]:
signature = infer_signature(X_test, y_test)

Logging the model:

In [ ]:
mlflow.sklearn.log_model("model", model_pipeline, signature=signature)

Let's finish the experiment

In [ ]:
mlflow.end_run()